In [1]:
import urllib2, time, random, re
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib

from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, UniqueConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import sqlalchemy

In [2]:
def search_allrecipes(searchstring):
    query = {"wt": searchstring}
    query = urllib.urlencode(query)
    url = str("http://allrecipes.com/search/results/?" + query)
    return url

In [3]:
def extract_allrecipes_urls(start_url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8','Connection': 'keep-alive'}
    recipeurls = []
    try:
        soup = BeautifulSoup(urllib2.urlopen(urllib2.Request(start_url, headers=hdr)).read())
        url_list = []
        for recipeurl in soup.find_all(href=re.compile("^/recipe/[0-9]+/.+/")):
            recipeurls.append(str("http://allrecipes.com" + recipeurl.attrs["href"]))
    except:
        print "Error, skipping."
    return list(set(recipeurls))

In [4]:
search_allrecipes("burger")

'http://allrecipes.com/search/results/?wt=burger'

In [5]:
seed = extract_allrecipes_urls("http://allrecipes.com/recipes/?grouping=all")

/home/andylane/anaconda2/envs/insight/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/andylane/anaconda2/envs/insight/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [6]:
urls = []
for item in seed:
    urls.extend(extract_allrecipes_urls(item))
urls = list(set(urls))

In [7]:
new_urls = current_urls

NameError: name 'current_urls' is not defined

In [8]:
i = 0
seed = ["http://allrecipes.com/recipes/96/salad/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201"]
master_url_list = []
new_urls = seed
while i <= 5:
    current_urls = new_urls
    new_urls = []
    for item in current_urls:
        new_urls.extend(extract_allrecipes_urls(item))
    master_url_list.extend(list(set(new_urls)))
    i = i+1
    print(str(i), str(len(master_url_list)), str(len(current_urls)))

('1', '46', '1')
('2', '479', '46')
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipping.
Error, skipp

In [9]:
len(set(current_urls))

152

In [10]:
# This is necessary to remove some non-recipe pages that the regex misses... total hack
for item in master_url_list:
    if item.find("photos") > 0:
        master_url_list.remove(item)

In [11]:
scrapeme = list(set(master_url_list))

In [12]:
item

'http://allrecipes.com/recipe/223377/heathers-updated-potato-salad/photos/871796/'

In [13]:
dbname = 'restaurants'
username = 'andylane'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://andylane@localhost/restaurants


In [14]:
Base = declarative_base()

In [15]:
class Recipe(Base):
    __tablename__ = 'recipes'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    url = Column(String, unique=True)
    calories = Column(Integer)
    fat = Column(Float)
    carbs = Column(Float)
    protein = Column(Float)
    cholesterol = Column(Float)
    sodium = Column(Float)
    servings = Column(Integer)
    #ingredients = 
    #__table_args__ = {'extend_existing': True}
    
    def __repr__(self):
        return "<Recipe(name='%s', url='%s')>" % (
            self.name, self.url)
    
class Ingredient(Base):
    __tablename__ = 'ingredients'
    id = Column(Integer, primary_key = True)
    ingredient = Column(String, nullable = False)
    recipe_id = Column(Integer, ForeignKey('recipes.id'))
    
    recipe = relationship(Recipe, back_populates = 'ingredients')
    #__table_args__ = {'extend_existing': True}
    def __repr__(self):
        return "<Ingredient(ingredient='%s')>" % self.ingredient

Recipe.ingredients = relationship("Ingredient", order_by=Ingredient.id, back_populates="recipe")

In [16]:
Base.metadata.create_all(engine)

In [17]:
Session = sessionmaker(bind=engine)
session = Session()

In [18]:
len(master_url_list)

1165

In [19]:
len(scrapeme)

750

In [22]:
session.query(Recipe).filter_by(url=str("http://allrecipes.com/recipe/214675/spicy-chipotle-grilled-shrimp/")).all()

[<Recipe(name='Spicy Chipotle Grilled Shrimp', url='http://allrecipes.com/recipe/214675/spicy-chipotle-grilled-shrimp/')>]

In [23]:
failedurls = []

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8','Connection': 'keep-alive'}

for url in scrapeme:
    session.rollback()
    if len(session.query(Recipe).filter_by(url=str(url)).all()) == 0:
        try:
            soup = BeautifulSoup(urllib2.urlopen(urllib2.Request(url, headers=hdr)).read())
            url = url
            name = soup.find_all("h1", itemprop="name")[0].get_text()
            fat = soup.find_all("li", itemprop="fatContent")[0].span.get_text()
            calories = soup.find_all("li", itemprop="calories")[0].span.get_text()
            carbs = soup.find_all("li", itemprop="carbohydrateContent")[0].span.get_text()
            protein = soup.find_all("li", itemprop="proteinContent")[0].span.get_text()
            cholesterol = soup.find_all("li", itemprop="cholesterolContent")[0].span.get_text()
            sodium = soup.find_all("li", itemprop="sodiumContent")[0].span.get_text()
            servings = soup.find_all("meta", itemprop="recipeYield")[0]["content"]
            ingredients = []
            for ingredient in soup.find_all("span", class_="recipe-ingred_txt added"):
                ingredients.append(ingredient.contents)
            session.add(Recipe(
                name=name, 
                fat=fat, 
                calories=calories,
                carbs=carbs, 
                protein=protein,
                cholesterol=cholesterol,
                sodium=sodium,
                servings=servings,
                url=url,
                ingredients=[Ingredient(ingredient = str(item)) for item in ingredients]
                               ))
            try:
                session.commit()
            except sqlalchemy.exc.DataError:
                print("Sanitize your data, man")
                session.rollback()
                failedurls.append(url)
            except sqlalchemy.exc.InvalidRequestError:
                print("probably not unique")
                session.rollback()
        except:
            print url
    else:
        print(str(url) + " already in database")

http://allrecipes.com/recipe/14304/black-bean-salad/
http://allrecipes.com/recipe/237349/caesar-potato-salad/
http://allrecipes.com/recipe/54117/individual-baked-eggs/ already in database
http://allrecipes.com/recipe/212381/kerrys-sweet-potato-latkes/
http://allrecipes.com/recipe/258788/baby-potato-salad/
http://allrecipes.com/recipe/237625/easy-fruit-salad/
http://allrecipes.com/recipe/27070/new-red-potato-salad/
http://allrecipes.com/recipe/137590/romaine-and-mandarin-orange-salad-wit/
http://allrecipes.com/recipe/143865/speedy-pear-and-pecan-salad/
http://allrecipes.com/recipe/42936/sallys-spinach-mashed-potatoes/
http://allrecipes.com/recipe/140979/chicken-pasta-salad-with-cashews-and/
http://allrecipes.com/recipe/215586/summer-fruit-salad-ii/
http://allrecipes.com/recipe/24353/dill-potato-salad/
http://allrecipes.com/recipe/215314/slow-cooker-baked-potatoes/ already in database
http://allrecipes.com/recipe/108082/quinoa-salad-with-dried-fruit-and-nuts/
http://allrecipes.com/recipe

In [26]:
scrapeme

['http://allrecipes.com/recipe/14304/black-bean-salad/',
 'http://allrecipes.com/recipe/237349/caesar-potato-salad/',
 'http://allrecipes.com/recipe/54117/individual-baked-eggs/',
 'http://allrecipes.com/recipe/212381/kerrys-sweet-potato-latkes/',
 'http://allrecipes.com/recipe/258788/baby-potato-salad/',
 'http://allrecipes.com/recipe/237625/easy-fruit-salad/',
 'http://allrecipes.com/recipe/27070/new-red-potato-salad/',
 'http://allrecipes.com/recipe/137590/romaine-and-mandarin-orange-salad-wit/',
 'http://allrecipes.com/recipe/143865/speedy-pear-and-pecan-salad/',
 'http://allrecipes.com/recipe/42936/sallys-spinach-mashed-potatoes/',
 'http://allrecipes.com/recipe/140979/chicken-pasta-salad-with-cashews-and/',
 'http://allrecipes.com/recipe/215586/summer-fruit-salad-ii/',
 'http://allrecipes.com/recipe/24353/dill-potato-salad/',
 'http://allrecipes.com/recipe/215314/slow-cooker-baked-potatoes/',
 'http://allrecipes.com/recipe/108082/quinoa-salad-with-dried-fruit-and-nuts/',
 'http:/

In [ ]:
session.rollback()